<a href="https://colab.research.google.com/github/maxsaldi0923/Portfolio/blob/main/NLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Inference With BERT

## For this homework, we will work on (NLI)[https://nlp.stanford.edu/projects/snli/].

The task is, give two sentences: a premise and a hypothesis, to classify the relation between them. We have three classes to describe this relationship.

1. Entailment: the hypothesis follows from the fact that the premise is true
2. Contradiction: the hypothesis contradicts the fact that the premise is true
3. Neutral: There is not relationship between premise and hypothesis

See below for examples

![snli_task](media/snli_task.jpeg)

### Prereqs

In [ ]:
! pip install transformers datasets tqdm

In [ ]:
# Imports for most of the notebook
import torch
from transformers import BertModel
from transformers import AutoTokenizer
from typing import Dict, List
import random
from tqdm.autonotebook import tqdm

In [ ]:
print(torch.cuda.is_available())
#device = torch.device("cpu")
# TODO: Uncomment the below line if you see True in the print statement
device = torch.device("cuda:0")

True


### First let's load the Stanford NLI dataset from the huggingface datasets hub using the datasets package

### Explore the dataset!

In [ ]:
from datasets import load_dataset
dataset = load_dataset("snli")
print("Split sizes (num_samples, num_labels):\n", dataset.shape)
print("\nExample:\n", dataset['train'][0])

Split sizes (num_samples, num_labels):
 {'test': (10000, 3), 'train': (550152, 3), 'validation': (10000, 3)}

Example:
 {'premise': 'A person on a horse jumps over a broken down airplane.', 'hypothesis': 'A person is training his horse for a competition.', 'label': 1}


Each example is a dictionary with the keys: (premise, hypothesis, label).

#### Data Fields
 - premise: a string used to determine the truthfulness of the hypothesis
 - hypothesis: a string that may be true, false, or whose truth conditions may not be knowable when compared to the premise
 - label: an integer whose value may be either 0, indicating that the hypothesis entails the premise, 1, indicating that the premise and hypothesis neither entail nor contradict each other, or 2, indicating that the hypothesis contradicts the premise.

## Create Train, Validation and Test sets

In [ ]:
from datasets import load_dataset
from collections import defaultdict

def get_snli(train=10000, validation=1000, test=1000):
    snli = load_dataset('snli')
    train_dataset = get_even_datapoints(snli['train'], train)
    validation_dataset = get_even_datapoints(snli['validation'], validation)
    test_dataset = get_even_datapoints(snli['test'], test)

    return train_dataset, validation_dataset, test_dataset

def get_even_datapoints(datapoints, n):
    random.seed(42)
    dp_by_label = defaultdict(list)
    for dp in tqdm(datapoints, desc='Reading Datapoints'):
        dp_by_label[dp['label']].append(dp)

    unique_labels = [0, 1, 2]

    split = n//len(unique_labels)

    result_datapoints = []

    for label in unique_labels:
        result_datapoints.extend(random.sample(dp_by_label[label], split))

    return result_datapoints

train_dataset, validation_dataset, test_dataset = get_snli()

Reading Datapoints:   0%|          | 0/550152 [00:00<?, ?it/s]

Reading Datapoints:   0%|          | 0/10000 [00:00<?, ?it/s]

Reading Datapoints:   0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
## sub set stats
from collections import Counter

# num sample stats
print(len(train_dataset), len(validation_dataset), len(test_dataset))

# label distribution
print(Counter([t['label'] for t in train_dataset]))
print(Counter([t['label'] for t in validation_dataset]))
print(Counter([t['label'] for t in test_dataset]))

# We have a perfectly balanced dataset

9999 999 999
Counter({0: 3333, 1: 3333, 2: 3333})
Counter({0: 333, 1: 333, 2: 333})
Counter({0: 333, 1: 333, 2: 333})


### We want a function to load samples from the huggingface dataset so that they can be batched and encoded for our model.

### Now let's reimplement our tokenizer using the huggingface tokenizer.

### Notice that our __call__ method (the one called when we call an instance of our class) takes both a premise batch and a hypothesis batch.
### The HuggingFace BERT tokenizer knows to join these with the special sentence seperator token between them. We let HuggingFace do most of the work here for making batches of tokenized and encoded sentences.

In [ ]:
# Nothing to do for this class!

class BatchTokenizer:
    """Tokenizes and pads a batch of input sentences."""

    def __init__(self, model_name='prajjwal1/bert-small'):
        """Initializes the tokenizer

        Args:
            pad_symbol (Optional[str], optional): The symbol for a pad. Defaults to "<P>".
        """
        self.hf_tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model_name = model_name

    def get_sep_token(self,):
        return self.hf_tokenizer.sep_token

    def __call__(self, prem_batch: List[str], hyp_batch: List[str]) -> List[List[str]]:
        """Uses the huggingface tokenizer to tokenize and pad a batch.

        We return a dictionary of tensors per the huggingface model specification.

        Args:
            batch (List[str]): A List of sentence strings

        Returns:
            Dict: The dictionary of token specifications provided by HuggingFace
        """
        # The HF tokenizer will PAD for us, and additionally combine
        # The two sentences deimited by the [SEP] token.
        enc = self.hf_tokenizer(
            prem_batch,
            hyp_batch,
            padding=True,
            return_token_type_ids=False,
            return_tensors='pt'
        )

        return enc


# HERE IS AN EXAMPLE OF HOW TO USE THE BATCH TOKENIZER
tokenizer = BatchTokenizer()
x = tokenizer(*[["this is the first premise", "This is the second premise"], ["This is first hypothesis", "This is the second hypothesis"]])
print(x)
tokenizer.hf_tokenizer.batch_decode(x["input_ids"])

{'input_ids': tensor([[  101,  2023,  2003,  1996,  2034, 18458,   102,  2023,  2003,  2034,
         10744,   102,     0],
        [  101,  2023,  2003,  1996,  2117, 18458,   102,  2023,  2003,  1996,
          2117, 10744,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


['[CLS] this is the first premise [SEP] this is first hypothesis [SEP] [PAD]',
 '[CLS] this is the second premise [SEP] this is the second hypothesis [SEP]']

### We can batch the train, validation, and test data, and then run it through the tokenizer

In [ ]:
def generate_pairwise_input(dataset: List[Dict]) -> (List[str], List[str], List[int]):
    """
    TODO: group all premises and corresponding hypotheses and labels of the datapoints
    a datapoint as seen earlier is a dict of premis, hypothesis and label
    """
    premises = []
    hypothesis = []
    labels = []
    for x in dataset:
        premises.append(x['premise'])
        hypothesis.append(x['hypothesis'])
        labels.append(x['label'])

    return premises, hypothesis, labels

In [ ]:
train_premises, train_hypotheses, train_labels = generate_pairwise_input(train_dataset)
validation_premises, validation_hypotheses, validation_labels = generate_pairwise_input(validation_dataset)
test_premises, test_hypotheses, test_labels = generate_pairwise_input(test_dataset)

In [ ]:
def chunk(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def chunk_multi(lst1, lst2, n):
    for i in range(0, len(lst1), n):
        yield lst1[i: i + n], lst2[i: i + n]

batch_size = 16

# Notice that since we use huggingface, we tokenize and
# encode in all at once!
tokenizer = BatchTokenizer()
train_input_batches = [b for b in chunk_multi(train_premises, train_hypotheses, batch_size)]
# Tokenize + encode
train_input_batches = [tokenizer(*batch) for batch in train_input_batches]

### Let's batch the labels, ensuring we get them in the same order as the inputs

In [ ]:
def encode_labels(labels: List[int]) -> torch.FloatTensor:
    """Turns the batch of labels into a tensor

    Args:
        labels (List[int]): List of all labels in the batch

    Returns:
        torch.FloatTensor: Tensor of all labels in the batch
    """
    return torch.LongTensor([int(l) for l in labels])


train_label_batches = [b for b in chunk(train_labels, batch_size)]
train_label_batches = [encode_labels(batch) for batch in train_label_batches]

### Now we implement the model. Notice the TODO and the optional TODO (read why you may want to do this one.)

In [ ]:
import torch
from torch import nn
from transformers import BertModel
from typing import Dict

class NLIClassifier(torch.nn.Module):
    def __init__(self, output_size: int, hidden_size: int, model_name='prajjwal1/bert-small'):
        super().__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size

        # Initialize BERT, which we use instead of a single embedding layer.
        self.bert = BertModel.from_pretrained(model_name)

        # TODO [OPTIONAL]: Updating all BERT parameters can be slow and memory intensive.
        # Freeze them if training is too slow. Notice that the learning
        # rate should probably be smaller in this case.
        # Uncommenting out the below 2 lines means only our classification layer will be updated.

        for param in self.bert.parameters():
            param.requires_grad = False

        self.bert_hidden_dimension = self.bert.config.hidden_size

        # TODO: Add an extra hidden layer in the classifier, projecting
        #      from the BERT hidden dimension to hidden size. Hint: torch.nn.Linear()

        self.hidden_layer = torch.nn.Linear(self.bert_hidden_dimension, self.hidden_size)

        # TODO: Add a relu nonlinearity to be used in the forward method
        #      https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html

        self.relu = torch.nn.ReLU()

        self.classifier = torch.nn.Linear(self.hidden_size, self.output_size)
        self.log_softmax = torch.nn.LogSoftmax(dim=2)
    def encode_text(
        self,
        symbols: Dict
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols BERT.
            Then, get CLS represenation.

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: CLS token embedding
        """
        # First we get the contextualized embedding for each input symbol
        # We no longer need an LSTM, since BERT encodes context and
        # gives us a single vector describing the sequence in the form of the [CLS] token.
        encoded_sequence = self.bert(**symbols)
        # TODO: Get the [CLS] token
        #      The BertModel output. See here: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
        #      and check the returns for the forward method.
        # We want to return a tensor of the form batch_size x 1 x bert_hidden_dimension
        # print(encoded_sequence.last_hidden_state.shape)
        # Return only the first token's embedding from the last_hidden_state. Hint: using list slices
        cls_token_embedding = encoded_sequence.last_hidden_state[:, 0, :]

        # We want to return a tensor of the form batch_size x 1 x bert_hidden_dimension
        # Add an extra dimension to make it batch_size x 1 x bert_hidden_dimension
        return cls_token_embedding.unsqueeze(1)

    def forward(
      self,
      symbols: Dict,
  ) -> torch.Tensor:
      """_summary_

      Args:
          symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

      Returns:
          torch.Tensor: _description_
      """
      encoded_sents = self.encode_text(symbols)
      output = self.hidden_layer(encoded_sents)
      output = self.relu(output)
      output = self.classifier(output)
      return self.log_softmax(output)

In [ ]:
# For making predictions at test time
def predict(model: torch.nn.Module, sents: torch.Tensor) -> List:
    logits = model(sents.to(device))
    return list(torch.Tensor.cpu(torch.argmax(logits, axis=2).squeeze()).numpy())

### Evaluation metrics: Macro F1

In [ ]:
import numpy as np
from numpy import sum as t_sum
from numpy import logical_and


def precision(predicted_labels, true_labels, which_label=1):
    """
    Precision is True Positives / All Positives Predictions
    """
    pred_which = np.array([pred == which_label for pred in predicted_labels])
    true_which = np.array([lab == which_label for lab in true_labels])
    denominator = t_sum(pred_which)
    if denominator:
        return t_sum(logical_and(pred_which, true_which))/denominator
    else:
        return 0.


def recall(predicted_labels, true_labels, which_label=1):
    """
    Recall is True Positives / All Positive Labels
    """
    pred_which = np.array([pred == which_label for pred in predicted_labels])
    true_which = np.array([lab == which_label for lab in true_labels])
    denominator = t_sum(true_which)
    if denominator:
        return t_sum(logical_and(pred_which, true_which))/denominator
    else:
        return 0.


def f1_score(
    predicted_labels: List[int],
    true_labels: List[int],
    which_label: int
):
    """
    F1 score is the harmonic mean of precision and recall
    """
    P = precision(predicted_labels, true_labels, which_label=which_label)
    R = recall(predicted_labels, true_labels, which_label=which_label)

    if P and R:
        return 2*P*R/(P+R)
    else:
        return 0.


def macro_f1(
    predicted_labels: List[int],
    true_labels: List[int],
    possible_labels: List[int],
    label_map=None
):
    converted_prediction = [label_map[int(x)] for x in predicted_labels] if label_map else predicted_labels
    scores = [f1_score(converted_prediction, true_labels, l) for l in possible_labels]
    # Macro, so we take the uniform avg.
    return sum(scores) / len(scores)

### Training loop.

In [ ]:
def training_loop(
    num_epochs,
    train_features,
    train_labels,
    dev_sents,
    dev_labels,
    optimizer,
    model,
):
    print("Training...")
    loss_func = torch.nn.NLLLoss()
    batches = list(zip(train_features, train_labels))
    random.shuffle(batches)
    for i in range(num_epochs):
        losses = []
        for features, labels in tqdm(batches):
            # Empty the dynamic computation graph
            optimizer.zero_grad()
            preds = model(features.to(device)).squeeze(1)
            loss = loss_func(preds, labels.to(device))
            # Backpropogate the loss through our model
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        print(f"epoch {i}, loss: {sum(losses)/len(losses)}")
        # Estimate the f1 score for the development set
        print("Evaluating dev...")
        all_preds = []
        all_labels = []
        for sents, labels in tqdm(zip(dev_sents, dev_labels), total=len(dev_sents)):
            pred = predict(model, sents)
            all_preds.extend(pred)
            all_labels.extend(list(labels.cpu().numpy()))

        dev_f1 =  macro_f1(all_preds, all_labels,possible_labels=set(all_labels))
        print(f"Dev F1 {dev_f1}")

    # Return the trained model
    return model

In [ ]:
# You can increase epochs if need be
epochs = 20

# TODO: Find a good learning rate and hidden size
LR = 0.0002
hidden_size = 256

possible_labels = set(train_labels)
model = NLIClassifier(output_size=len(possible_labels), hidden_size=hidden_size)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), LR)

batch_tokenizer = BatchTokenizer()

validation_input_batches = [b for b in chunk_multi(validation_premises, validation_hypotheses, batch_size)]

# Tokenize + encode
validation_input_batches = [batch_tokenizer(*batch) for batch in validation_input_batches]
validation_batch_labels = [b for b in chunk(validation_labels, batch_size)]
validation_batch_labels = [encode_labels(batch) for batch in validation_batch_labels]

training_loop(
    epochs,
    train_input_batches,
    train_label_batches,
    validation_input_batches,
    validation_batch_labels,
    optimizer,
    model,
)

Training...


  0%|          | 0/625 [00:00<?, ?it/s]

epoch 0, loss: 1.0851078232765197
Evaluating dev...


  0%|          | 0/63 [00:00<?, ?it/s]

Dev F1 0.4771621641386233


...

  0%|          | 0/625 [00:00<?, ?it/s]

epoch 19, loss: 0.6763649537324905
Evaluating dev...


  0%|          | 0/63 [00:00<?, ?it/s]

Dev F1 0.5600793164785816


NLIClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-3): 4 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affin

In [ ]:
# TODO: Get a final macro F1 on the test set.
# You should be able to mimic what we did with the validaiton set.


Dev F1 0.5161333030636731

## Evaluation of Pretrained Model

Huggingface also hosts models which users have already trained on SNLI -- we can load them here and evaluate their performance on the validation and test set.

These models include the BertModel which we were using before, but also the trained weights for the classifier layer. Because of this, we'll use the standard HuggingFace classification model instead of the classifier we used above, and modify the training and prediction functions to handle this correctly.

Try and find the differences between the training loop. One addition is the new usage of "label_map". Why may this be necessary?

In [ ]:
def class_predict(model, sents):

    with torch.inference_mode():

        logits = model(**sents.to(device)).logits
        predictions = torch.argmax(logits, axis=1)

    return predictions

def prediction_loop(model, dev_sents, dev_labels, label_map=None):
    print("Evaluating...")
    all_preds = []
    all_labels = []
    for sents, labels in tqdm(zip(dev_sents, dev_labels), total=len(dev_sents)):
        pred = class_predict(model, sents).cpu()
        all_preds.extend(pred)
        all_labels.extend(list(labels.cpu().numpy()))

    dev_f1 = macro_f1(all_preds, all_labels, possible_labels=set(all_labels), label_map = label_map)
    print(f"F1 {dev_f1}")

def class_training_loop(
    num_epochs,
    train_features,
    train_labels,
    dev_sents,
    dev_labels,
    optimizer,
    model,
    label_map=None
):
    print("Training...")
    loss_func = torch.nn.CrossEntropyLoss()
    batches = list(zip(train_features, train_labels))
    random.shuffle(batches)
    for i in range(num_epochs):
        losses = []
        for features, labels in tqdm(batches):
            # Empty the dynamic computation graph
            optimizer.zero_grad()
            logits = model(**features.to(device)).logits
            # preds = torch.argmax(logits, axis=1)
            loss = loss_func(logits, labels)
            # Backpropogate the loss through our model
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        print(f"epoch {i}, loss: {sum(losses)/len(losses)}")
        # Estimate the f1 score for the development set
    print("Evaluating dev...")
    all_preds = []
    all_labels = []
    for sents, labels in tqdm(zip(dev_sents, dev_labels), total=len(dev_sents)):
        pred = predict(model, sents).cpu()
        all_preds.extend(pred)
        all_labels.extend(list(labels.numpy()))

    all_preds
    dev_f1 = macro_f1(all_preds, all_labels, possible_labels=set(all_labels), label_map = label_map)
    print(f"Dev F1 {dev_f1}")

    # Return the trained model
    return model

Now we can load a model and re-tokenize our data.

In [ ]:
# TODO: Get the label_map
# For the snli dataset 0=Entailment, 1=Neutral, 2=Contradiction
label_map = {0:2, 1:0, 2:1}

In [ ]:
from transformers import  BertForSequenceClassification, BertTokenizer


model_name = 'textattack/bert-base-uncased-snli'
tokenizer_model_name = 'textattack/bert-base-uncased-snli' # This is sometimes different from model_name, but should normally be the same

model =  BertForSequenceClassification.from_pretrained(model_name)

model.to(device)

batch_tokenizer = BatchTokenizer(model_name = tokenizer_model_name)

validation_input_batches = [b for b in chunk_multi(validation_premises, validation_hypotheses, batch_size)]

# Tokenize + encode
validation_input_batches = [batch_tokenizer(*batch) for batch in validation_input_batches]
validation_batch_labels = [b for b in chunk(validation_labels, batch_size)]
validation_batch_labels = [encode_labels(batch) for batch in validation_batch_labels]
prediction_loop(model, validation_input_batches, validation_batch_labels, label_map=label_map)

Evaluating...


  0%|          | 0/63 [00:00<?, ?it/s]

F1 0.6671987159037203


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from sklearn.metrics import classification_report
import random

# Load the development set
dataset = load_dataset("snli", split="validation")
dev_premises = dataset['premise']
dev_hypotheses = dataset['hypothesis']
dev_labels = dataset['label']

# Define a function for evaluating a BERT-based model
def evaluate_bert_model(model_name, tokenizer_model_name):
    # Load model and tokenizer
    model = BertForSequenceClassification.from_pretrained(model_name)
    tokenizer = BertTokenizer.from_pretrained(tokenizer_model_name)

    # Select a random subset of the validation set
    sample_size = 100
    sampled_indices = random.sample(range(len(dev_premises)), sample_size)
    sampled_premises = [dev_premises[i] for i in sampled_indices]
    sampled_hypotheses = [dev_hypotheses[i] for i in sampled_indices]
    sampled_labels = [dev_labels[i] for i in sampled_indices]



    # Tokenize and encode the sampled data
    inputs = tokenizer(sampled_premises, sampled_hypotheses, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits

    # Predicted labels
    predicted_labels = logits.argmax(dim=1).tolist()

    # Print classification report
    print(f"Evaluation report for model: {model_name}")
    print(classification_report(sampled_labels, predicted_labels))




label_map = {0:2, 1:0, 2:1}
# Models to evaluate
model1_name = 'bvanaken/clinical-assertion-negation-bert'
tokenizer1_model_name = 'bvanaken/clinical-assertion-negation-bert'


model2_name = 'nlpaueb/legal-bert-base-uncased'
tokenizer2_model_name = 'nlpaueb/legal-bert-base-uncased'

# Evaluate the models
evaluate_bert_model(model1_name, tokenizer1_model_name)
evaluate_bert_model(model2_name, tokenizer2_model_name)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Evaluation report for model: bvanaken/clinical-assertion-negation-bert
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         3
           0       0.34      1.00      0.51        34
           1       0.00      0.00      0.00        34
           2       0.00      0.00      0.00        29

    accuracy                           0.34       100
   macro avg       0.09      0.25      0.13       100
weighted avg       0.12      0.34      0.17       100



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Some weights of BertForSequenceClassification were not initializ

Evaluation report for model: nlpaueb/legal-bert-base-uncased
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         4
           0       0.45      0.83      0.58        36
           1       0.27      0.35      0.31        26
           2       0.00      0.00      0.00        34

    accuracy                           0.39       100
   macro avg       0.18      0.29      0.22       100
weighted avg       0.23      0.39      0.29       100



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


To complete this section, find 2 other BERT-based models which have been trained on natural language inference and evaluate them on the dev set. Note the scores for each model and any high-level differences you can find between the models (architecture, sizes, training data, etc. )

If you don't have access to a GPU, inference may be slow, particularly for larger models. In this case, take a sample of the validation set; the size should be large enough such that all labels are covered, and a score will still be meaningful, but also so that inference doesn't take more than 3-5 minutes.

n evaluating three BERT-based models for natural language inference, the original model (textattack/bert-base-uncased-snli) showed a relatively high F1 score of around 0.667. Comparatively, the clinical-assertion-negation-bert model achieved a similar F1 score, with precision concentrated in class 0 (Neutral). On the other hand, the legal-bert-base-uncased model yielded a lower F1 score of about 0.39, indicating challenges in classifying instances related to classes -1 and 2. The analysis of precision, recall, and F1 scores highlighted distinct weaknesses in predicting specific classes for each model. These findings suggest opportunities for targeted improvements, such as fine-tuning or exploring alternative architectures, to enhance overall model performance.

# Written Assignment

### 1. Describe the task and what capability is required to solve it.

### 2. How does the method of encoding sequences of words in our model differ here, compared to the word embeddings in HW 4. What is different? Why benefit does this method have?

### 3. Discuss your results. Did you do any hyperparameter tuning? Did the model solve the task?

1).

The task involves classifying the relationship between two sentences, represented as a premise and a hypothesis, into one of three classes: Entailment, Contradiction, or Neutral. In Entailment, the hypothesis logically follows from the truth of the premise. Contradiction denotes a situation where the hypothesis contradicts the truth of the premise. The Neutral class signifies that there is no discernible relationship between the premise and the hypothesis. The required capability to solve this task is the ability to understand the contextual and logical connections between pairs of sentences, recognizing entailment, contradiction, or neutrality based on the provided definitions. Additionally, the model should be capable of tokenizing, encoding, and classifying large sets of sentence pairs efficiently, with evaluation based on the Macro F1 metric to assess overall classification performance.

2).
The encoding method in the natural language inference (NLI) model, utilizing BERT, differs significantly from the word embeddings employed in the POS-Tagger and Named Entity Recognition (NER) models for Hindi datasets. Unlike static word embeddings used in POS-Tagger and NER, BERT provides contextualized embeddings, capturing the nuanced dependencies and meanings of words within the entire sentence. The bidirectional attention mechanism in BERT allows it to consider both left and right contexts during training, enabling a more comprehensive understanding of word relationships. Additionally, BERT's fine-tuning capability on pre-trained language understanding tasks facilitates adaptation to specific downstream tasks. This approach contrasts with traditional models, enhancing the NLI model's ability to capture complex contextual information and achieve superior performance in tasks requiring nuanced comprehension of sentence pairs.

3).
In the process of developing the natural language inference (NLI) model, extensive hyperparameter tuning was performed to enhance its performance. Despite these efforts, the achieved F1 score of 0.56 on the development set suggests that the model may not have fully solved the task. The relatively moderate F1 score indicates room for improvement in capturing the relationships between premises and hypotheses accurately. Further exploration of hyperparameter configurations, such as learning rates, hidden sizes, or incorporating additional architectural modifications, might be necessary to boost the model's efficacy. Additionally, conducting a thorough analysis of misclassifications and adjusting the training strategy accordingly could contribute to refining the model's capabilities and addressing specific challenges within the task.








In [1]:
print("hello")

hello
